In [1]:
import pickle
import pandas as pd
import os
import cv2
import torch

# GPU 사용이 가능할 경우, GPU를 사용할 수 있게 함.'
os.environ['CUDA_VISIBLE_DEVICES'] = '2'
device = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device(device)
print(device)

print(os.environ.get('CUDA_VISIBLE_DEVICES'))


cuda
2


/home/MMI24limbyungwan/.conda/envs/byungwan_resn/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

N_CLASSES = 13
# 데이터셋을 불러옵니다.
csv_file = os.path.join("/mnt/nas27/Dataset/Samsung_DM", './train_source.csv')
data = pd.read_csv(csv_file)

# 클래스별 샘플 수를 저장할 리스트를 초기화합니다.
class_sample_counts = [0] * N_CLASSES  # num_classes는 클래스의 총 개수입니다.

# 데이터셋을 순회하면서 클래스별 샘플 수를 세고 저장합니다.
for idx in range(len(data)):
    mask_path = os.path.join("/mnt/nas27/Dataset/Samsung_DM", data.iloc[idx, 2][2:])  # 마스크 이미지 경로
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    mask[mask == 255] = 12  # 배경 클래스로 처리

    # 각 클래스의 샘플 수를 카운트합니다.
    for class_id in range(N_CLASSES):
        class_sample_counts[class_id] += (mask == class_id).sum()

# 클래스별 가중치를 계산합니다.
total_samples = sum(class_sample_counts)
class_weights = [total_samples / count for count in class_sample_counts]

# 클래스별 가중치를 텐서로 변환합니다.
class_weights = torch.tensor(class_weights).to(device)
class_weights = class_weights.float()


# class_weights를 저장할 파일 경로
class_weights_path = 'class_weights.pkl'

# class_weights를 파일로 저장
with open(class_weights_path, 'wb') as file:
    pickle.dump(class_weights, file)


In [3]:
print(class_weights)

tensor([   6.2232,   81.0473,    5.1017,   66.2180,  170.2419,  506.2671,
         159.9452,    6.9654,    4.3830,  907.1918, 2223.4863,   10.2576,
           7.6339], device='cuda:0')
